<a href="https://colab.research.google.com/github/Murcha1990/AppliedDS_DPO_2024/blob/main/Lesson2_NLP2/AiEdu_CharRNN_solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Character-Level LSTM
На этом занятии поговорим про рекуррентные нейронные сети (Recurrent Neural Networ, RNN). Мы обучим модель на тексте книги "Анна Каренина", после чего попробуем генерировать новый текст.

**Модель сможет генерировать новый текст на основе текста "Анны Карениной"!**

Можно посмотреть полезную [статью про RNNs](http://karpathy.github.io/2015/05/21/rnn-effectiveness/) и [реализацию в Torch](https://github.com/karpathy/char-rnn).

Ообщая архитектура RNN:

<img src="https://github.com/udacity/deep-learning-v2-pytorch/blob/master/recurrent-neural-networks/char-rnn/assets/charseq.jpeg?raw=1" width="500">

In [ ]:
import numpy as np
import torch
import torch.nn.functional as F
from torch import nn

## Загрузим данные

Загрузим текстовый файл.

In [ ]:
!wget !wget https://raw.githubusercontent.com/aiedu-courses/rnn_bootcamp/main/shakespeare.txt

--2024-01-14 11:22:13--  http://!wget/
Resolving !wget (!wget)... failed: Name or service not known.
wget: unable to resolve host address ‘!wget’
--2024-01-14 11:22:13--  https://raw.githubusercontent.com/aiedu-courses/rnn_bootcamp/main/shakespeare.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘shakespeare.txt.1’

shakespeare.txt.1   100%[===================>]   1.06M  --.-KB/s    in 0.02s   

2024-01-14 11:22:13 (58.2 MB/s) - ‘shakespeare.txt.1’ saved [1115394/1115394]

FINISHED --2024-01-14 11:22:13--
Total wall clock time: 0.2s
Downloaded: 1 files, 1.1M in 0.02s (58.2 MB/s)


In [ ]:
# open text file and read in data as `text`
with open("shakespeare.txt", "r") as f:
    text = f.read()

Посмотрим первые 100 символов:

In [ ]:
text[:100]

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'

### Токенизация

В ячейках ниже создадим два **словаря** для преобразования символов в целые числа и обратно. Кодирование символов как целых чисел упрощает их использование в качестве входных данных в сети.

In [ ]:
# encode the text and map each character to an integer and vice versa

# we create two dictionaries:
# 1. int2char, which maps integers to characters
# 2. char2int, which maps characters to unique integers
chars = tuple(set(text))

int2char = dict(enumerate(chars))
char2int = {ch: ii for ii, ch in int2char.items()}

# encode the text
encoded = np.array([char2int[ch] for ch in text])

Посмотрим как символы закодировались целыми числами

In [ ]:
encoded[:100]

array([23, 41, 49, 35, 37,  9, 26, 41, 37, 41, 57, 58, 44, 12, 31, 30, 58,
       40, 13, 49, 58,  9, 63, 58,  9, 50, 49, 13,  8, 58, 58, 27,  9, 33,
       44, 34,  9, 40, 39, 49, 37,  6, 58, 49, 21,  9,  6, 58, 33, 49,  9,
       42, 58,  9, 35, 50, 58, 33, 18, 61, 31, 31,  7, 48, 48, 12, 31, 22,
       50, 58, 33, 18, 21,  9, 35, 50, 58, 33, 18, 61, 31, 31, 23, 41, 49,
       35, 37,  9, 26, 41, 37, 41, 57, 58, 44, 12, 31, 59, 13, 39])

## Предобработка данных

Как можно видеть на изображении char-RNN выше, сеть ожидает **one-hot encoded** входа, что означает, что каждый символ преобразуется в целое число (через созданный словарь), а затем преобразуется в вектор-столбец, где только соответствующий ему целочисленный индекс будет иметь значение 1, а остальная часть вектора будет заполнена нулями. Давайте создадим для этого функцию.

In [ ]:
def one_hot_encode(arr, n_labels):

    # Initialize the the encoded array
    one_hot = np.zeros((arr.size, n_labels), dtype=np.float32)

    # Fill the appropriate elements with ones
    one_hot[np.arange(one_hot.shape[0]), arr.flatten()] = 1.0

    # Finally reshape it to get back to the original array
    one_hot = one_hot.reshape((*arr.shape, n_labels))

    return one_hot

In [ ]:
# check that the function works as expected
test_seq = np.array([[3, 5, 1]])
one_hot = one_hot_encode(test_seq, 8)

print(one_hot)

[[[0. 0. 0. 1. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 1. 0. 0.]
  [0. 1. 0. 0. 0. 0. 0. 0.]]]


## Создаем мини-батчи (mini-batchs)


Создатдим мини-батчи для обучения. На простом примере они будут выглядеть так:

<img src="https://github.com/udacity/deep-learning-v2-pytorch/blob/master/recurrent-neural-networks/char-rnn/assets/sequence_batching@1x.png?raw=1" width=500px>
<br>

Возьмем закодированные символы (переданные как параметр `arr`) и разделим их на несколько последовательностей, заданных параметром `batch_size`. Каждая из наших последовательностей будет иметь длину `seq_length`.

### Создани батчей

**1. Первое, что нам нужно сделать, это отбросить часть текста, чтобы у нас были только полные мини-батчи**

Каждый батч содержит $ N\times M $ символов, где $ N $ - это размер батча (количество последовательностей в батче), а $ M $ - длина `seq_length` или количество шагов в последовательности. Затем, чтобы получить общее количество батчей $ K $, которое мы можем сделать из массива `arr`, нужно разделить длину `arr` на количество символов в батче. Когда мы узнаем количество батчей, можно получить общее количество символов, которые нужно сохранить, из `arr`: $ N * M * K $.

**2. После этого нам нужно разделить `arr` на $N$ батчей**

Это можно сделать с помощью `arr.reshape(size)`, где `size` - это кортеж, содержащий размеры измененного массива. Мы знаем, что нам нужно $ N $ последовательностей в батче, поэтому сделаем его размером первого измерения. Для второго измерения можем использовать «-1» в качестве заполнителя, он заполнит массив соответствующими данными. После этого должен остаться массив $N\times(M * K)$.

**3. Теперь, когда у нас есть этот массив, мы можем перебирать его, чтобы получить наши мини-батчи**

Идея состоит в том, что каждая партия представляет собой окно $ N\times M $ в массиве $ N\times (M * K) $. Для каждого последующего батча окно перемещается на `seq_length`. Мы также хотим создать как входной, так и выходной массивы. Это окно можно сделать с помощью `range`, чтобы делать шаги размером `n_steps` от $ 0 $ до `arr.shape [1]`, общее количество токенов в каждой последовательности. Таким образом, целые числа, которые получены из диапазона, всегда указывают на начало батча, и каждое окно имеет ширину `seq_length`.

In [ ]:
def get_batches(int_words, batch_size, seq_length):
    """
    Generates batches from encoded sequence.
    :param int_words: tensor of ints, which represents the text; shape: [batch_size, -1]
    :param batch_size: number of sequences per batch
    :param seq_length: number of encoded chars in a sequence
    :return: generator of pairs (x, y); x_shape, y_shape: [batch_size, seq_length - 1]
    """
    # 1. Truncate text, so there are only full batches
    window_size = seq_length + 1
    batch_size_total = batch_size * window_size
    n_batches = len(int_words) // batch_size_total
    int_words = int_words[: n_batches * batch_size_total]

    # 2. Reshape into batch_size rows
    int_words = int_words.reshape((batch_size, -1))

    # 3. Iterate through the text matrix
    for position in range(0, int_words.shape[1], window_size):
        x = int_words[:, position : position + window_size - 1]
        y = int_words[:, position + 1 : position + window_size]
        yield x, y

### Протестируем

Теперь создадим несколько наборов данных, и проверим, что происходит, когда мы создаем батчи.

In [ ]:
batches = get_batches(encoded, 8, 50)
x, y = next(batches)

In [ ]:
# printing out the first 10 items in a sequence
print("x\n", x[:10, :10])
print("\ny\n", y[:10, :10])

x
 [[23 41 49 35 37  9 26 41 37 41]
 [ 6  9 37  6 34  9 36 41  8 37]
 [ 9 41 35 12 31 30 58 37 37 58]
 [33 44  9 33 48 58  6 13 39 35]
 [ 8 41 33 44 12 31 60  6 33 37]
 [33 49 51 58 19 31 60 58  9 63]
 [58 48  9 33 44 27  9 14  9 18]
 [ 9 37  6 33 37  9 13 44  8 58]]

y
 [[41 49 35 37  9 26 41 37 41 57]
 [ 9 37  6 34  9 36 41  8 37 13]
 [41 35 12 31 30 58 37 37 58 49]
 [44  9 33 48 58  6 13 39 35 58]
 [41 33 44 12 31 60  6 33 37  9]
 [49 51 58 19 31 60 58  9 63 41]
 [48  9 33 44 27  9 14  9 18 44]
 [37  6 33 37  9 13 44  8 58  9]]


Если вы правильно реализовали get_batches, результат должен выглядеть примерно так:
```
x
 [[25  8 60 11 45 27 28 73  1  2]
 [17  7 20 73 45  8 60 45 73 60]
 [27 20 80 73  7 28 73 60 73 65]
 [17 73 45  8 27 73 66  8 46 27]
 [73 17 60 12 73  8 27 28 73 45]
 [66 64 17 17 46  7 20 73 60 20]
 [73 76 20 20 60 73  8 60 80 73]
 [47 35 43  7 20 17 24 50 37 73]]

y
 [[ 8 60 11 45 27 28 73  1  2  2]
 [ 7 20 73 45  8 60 45 73 60 45]
 [20 80 73  7 28 73 60 73 65  7]
 [73 45  8 27 73 66  8 46 27 65]
 [17 60 12 73  8 27 28 73 45 27]
 [64 17 17 46  7 20 73 60 20 80]
 [76 20 20 60 73  8 60 80 73 17]
 [35 43  7 20 17 24 50 37 73 36]]
 ```
 хотя точные цифры могут отличаться. Убедитесь, что данные сдвинуты на один шаг для `y`!!!

---
## Зададим архитектуру


<img src="https://github.com/udacity/deep-learning-v2-pytorch/blob/master/recurrent-neural-networks/char-rnn/assets/charRNN.png?raw=1" width=500px>

### Структура модели

В `__init__` предлагаемая структура выглядит следующим образом:
* Создаваём и храним необходимые словари (уже релизовано)
* Определяем слой LSTM, который принимает в качестве параметров: размер ввода (количество символов), размер скрытого слоя `n_hidden`, количество слоев` n_layers`, вероятность drop-out'а `drop_prob` и логическое значение batch_first (True)
* Определяем слой drop-out с помощью drop_prob
* Определяем полносвязанный слой с параметрами: размер ввода `n_hidden` и размер выхода - количество символов
* Наконец, инициализируем веса

Обратите внимание, что некоторые параметры были названы и указаны в функции `__init__`, их нужно сохранить и использовать, выполняя что-то вроде` self.drop_prob = drop_prob`.

---
### Входы-выходы LSTM

Вы можете создать [LSTM layer](https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html#torch.nn.LSTM) следующим образом

```python
self.lstm = nn.LSTM(input_size, n_hidden, n_layers,
                            dropout=drop_prob, batch_first=True)
```

где `input_siz`e - это количество символов, которые эта ячейка ожидает видеть в качестве последовательного ввода, а `n_hidden` - это количество элементов в скрытых слоях ячейки. Можно добавить drop-out, добавив параметр `dropout` с заданной вероятностью. Наконец, в функции `forward` мы можем складывать ячейки LSTM в слои, используя `.view`.

Также требуется создать начальное скрытое состояние всех нулей:

```python
self.init_hidden()
```

In [ ]:
# check if GPU is available
train_on_gpu = torch.cuda.is_available()

if train_on_gpu:
    print("Training on GPU!")
else:
    print("No GPU available, training on CPU; consider making n_epochs very small.")

Training on GPU!


In [ ]:
class CharRNN(nn.Module):
    def __init__(self, tokens, n_hidden=256, n_layers=2, drop_prob=0.5, lr=0.001):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr

        # creating character dictionaries
        self.chars = tokens
        self.int2char = dict(enumerate(self.chars))
        self.char2int = {ch: ii for ii, ch in self.int2char.items()}

        ## TODO: define the LSTM
        self.lstm = nn.LSTM(
            len(self.chars), n_hidden, n_layers, dropout=drop_prob, batch_first=True
        )

        ## TODO: define a dropout layer
        self.dropout = nn.Dropout(drop_prob)

        ## TODO: define the final, fully-connected output layer
        self.fc = nn.Linear(n_hidden, len(self.chars))

    def forward(self, x, hidden):
        """Forward pass through the network.
        These inputs are x, and the hidden/cell state `hidden`."""

        ## TODO: Get the outputs and the new hidden state from the lstm
        r_output, hidden = self.lstm(x, hidden)

        ## TODO: pass through a dropout layer
        out = self.dropout(r_output)

        # Stack up LSTM outputs using view
        # you may need to use contiguous to reshape the output
        out = out.contiguous().view(-1, self.n_hidden)

        ## TODO: put x through the fully-connected layer
        out = self.fc(out)

        # return the final output and the hidden state
        return out, hidden

    def init_hidden(self, batch_size):
        """Initializes hidden state"""
        # Create two new tensors with sizes n_layers x batch_size x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data

        if train_on_gpu:
            hidden = (
                weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda(),
                weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda(),
            )
        else:
            hidden = (
                weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),
                weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),
            )

        return hidden

## Обучим модель

Во время обучения нужно установить количество эпох, скорость обучения и другие параметры.

Используем оптимизатор Adam и кросс-энтропию, считаем loss и, как обычно, выполняем back propagation.

Пара подробностей об обучении:
> * Во время цикла мы отделяем скрытое состояние от его истории; на этот раз установив его равным новой переменной * tuple *, потому что скрытое состояние LSTM, является кортежем скрытых состояний.
* Мы используем [`clip_grad_norm_`](https://pytorch.org/docs/stable/_modules/torch/nn/utils/clip_grad.html) чтобы избавиться от взрывающихся градиентов.

In [ ]:
def train(
    net,
    data,
    epochs=10,
    batch_size=10,
    seq_length=50,
    lr=0.001,
    clip=5,
    val_frac=0.1,
    print_every=10,
):
    """Training a network

    Arguments
    ---------

    net: CharRNN network
    data: text data to train the network
    epochs: Number of epochs to train
    batch_size: Number of mini-sequences per mini-batch, aka batch size
    seq_length: Number of character steps per mini-batch
    lr: learning rate
    clip: gradient clipping
    val_frac: Fraction of data to hold out for validation
    print_every: Number of steps for printing training and validation loss

    """
    net.train()

    opt = torch.optim.Adam(net.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()

    # create training and validation data
    val_idx = int(len(data) * (1 - val_frac))
    data, val_data = data[:val_idx], data[val_idx:]

    if train_on_gpu:
        net.cuda()

    counter = 0
    n_chars = len(net.chars)
    for e in range(epochs):
        # initialize hidden state
        h = net.init_hidden(batch_size)

        for x, y in get_batches(data, batch_size, seq_length):
            counter += 1

            # One-hot encode our data and make them Torch tensors
            x = one_hot_encode(x, n_chars)
            inputs, targets = torch.from_numpy(x), torch.from_numpy(y)

            if train_on_gpu:
                inputs, targets = inputs.cuda(), targets.cuda()

            # Creating new variables for the hidden state, otherwise
            # we'd backprop through the entire training history
            h = tuple([each.data for each in h])

            # zero accumulated gradients
            net.zero_grad()

            # get the output from the model
            output, h = net(inputs, h)

            # calculate the loss and perform backprop
            loss = criterion(output, targets.view(batch_size * seq_length).long())
            loss.backward()
            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
            nn.utils.clip_grad_norm_(net.parameters(), clip)
            opt.step()

            # loss stats
            if counter % print_every == 0:
                # Get validation loss
                val_h = net.init_hidden(batch_size)
                val_losses = []
                net.eval()
                for x, y in get_batches(val_data, batch_size, seq_length):
                    # One-hot encode our data and make them Torch tensors
                    x = one_hot_encode(x, n_chars)
                    x, y = torch.from_numpy(x), torch.from_numpy(y)

                    # Creating new variables for the hidden state, otherwise
                    # we'd backprop through the entire training history
                    val_h = tuple([each.data for each in val_h])

                    inputs, targets = x, y
                    if train_on_gpu:
                        inputs, targets = inputs.cuda(), targets.cuda()

                    output, val_h = net(inputs, val_h)
                    val_loss = criterion(
                        output, targets.view(batch_size * seq_length).long()
                    )

                    val_losses.append(val_loss.item())

                net.train()  # reset to train mode after iterationg through validation data

                print(
                    "Epoch: {}/{}...".format(e + 1, epochs),
                    "Step: {}...".format(counter),
                    "Loss: {:.4f}...".format(loss.item()),
                    "Val Loss: {:.4f}".format(np.mean(val_losses)),
                )

## Определим модель

Теперь мы можем создать модель с заданными гиперпараметрами. Определим размеры мини-батчей.

In [ ]:
# define and print the net
n_hidden = 512
n_layers = 2

net = CharRNN(chars, n_hidden, n_layers)
print(net)

CharRNN(
  (lstm): LSTM(65, 512, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=512, out_features=65, bias=True)
)


In [ ]:
s = 0

for layer in net.parameters():
    s += layer.numel()

s

3320385

### Установим гиперпараметры

In [ ]:
batch_size = 128
seq_length = 100
n_epochs = 20  # start smaller if you are just testing initial behavior

# train the model
train(
    net,
    encoded,
    epochs=n_epochs,
    batch_size=batch_size,
    seq_length=seq_length,
    lr=1,
    print_every=10,
)

Epoch: 1/20... Step: 10... Loss: 879.7095... Val Loss: 952.9646
Epoch: 1/20... Step: 20... Loss: 2173.8604... Val Loss: 2263.5418
Epoch: 1/20... Step: 30... Loss: 2184.7051... Val Loss: 2162.5851
Epoch: 1/20... Step: 40... Loss: 1521.5664... Val Loss: 1393.3106
Epoch: 1/20... Step: 50... Loss: 953.1794... Val Loss: 985.6353
Epoch: 1/20... Step: 60... Loss: 628.0311... Val Loss: 671.9441
Epoch: 1/20... Step: 70... Loss: 582.3876... Val Loss: 586.3158
Epoch: 2/20... Step: 80... Loss: 376.1260... Val Loss: 372.7719
Epoch: 2/20... Step: 90... Loss: 359.9218... Val Loss: 335.3066
Epoch: 2/20... Step: 100... Loss: 309.0128... Val Loss: 317.7114
Epoch: 2/20... Step: 110... Loss: 302.7938... Val Loss: 315.5639
Epoch: 2/20... Step: 120... Loss: 307.3758... Val Loss: 274.2639
Epoch: 2/20... Step: 130... Loss: 316.4543... Val Loss: 281.6413
Epoch: 2/20... Step: 140... Loss: 393.9493... Val Loss: 369.6179
Epoch: 2/20... Step: 150... Loss: 291.6541... Val Loss: 234.8034
Epoch: 3/20... Step: 160... 

## Checkpoint

После обучения сохраним модель, чтобы можно было загрузить ее позже. Здесь сохраняются параметры, необходимые для создания той же архитектуры, гиперпараметры скрытого слоя и токены.

In [ ]:
# change the name, for saving multiple files
model_name = "rnn_x_epoch.net"

checkpoint = {
    "n_hidden": net.n_hidden,
    "n_layers": net.n_layers,
    "state_dict": net.state_dict(),
    "tokens": net.chars,
}

with open(model_name, "wb") as f:
    torch.save(checkpoint, f)

---
## Делаем предсказания

Теперь, когда мы обучили модель, сделаем предсказание следующих символов! Для предсказания мы передаем последний символ, и сеть предсказывает следующий символ, который мы потом передаем снова на вхол и получаем еще один предсказанный символ и так далее...

Наши прогнозы основаны на категориальном распределении вероятностей по всем возможным символам. Мы можем ограничить число символов, чтобы сделать получаемый предсказанный текст более разумным, рассматривая только некоторые наиболее вероятные символы $K$. Это не позволит сети выдавать нам совершенно абсурдные прогнозы, а также позволит внести некоторый шум и случайность в выбранный текст. Узнать больше [можно здесь](https://pytorch.org/docs/stable/generated/torch.topk.html#torch.topk).

In [ ]:
def predict(net, char, h=None, top_k=None):
    """Given a character, predict the next character.
    Returns the predicted character and the hidden state.
    """

    # tensor inputs
    x = np.array([[net.char2int[char]]])
    x = one_hot_encode(x, len(net.chars))
    inputs = torch.from_numpy(x)

    if train_on_gpu:
        inputs = inputs.cuda()

    # detach hidden state from history
    h = tuple([each.data for each in h])
    # get the output of the model
    out, h = net(inputs, h)

    # get the character probabilities
    p = F.softmax(out, dim=1).data
    if train_on_gpu:
        p = p.cpu()  # move to cpu

    # get top characters
    if top_k is None:
        top_ch = np.arange(len(net.chars))
    else:
        p, top_ch = p.topk(top_k)
        top_ch = top_ch.numpy().squeeze()

    # select the likely next character with some element of randomness
    p = p.numpy().squeeze()
    char = np.random.choice(top_ch, p=p / p.sum())

    # return the encoded value of the predicted char and the hidden state
    return net.int2char[char], h

### Priming и генерирование текста

Нужно задать скрытое состояние, чтобы сеть не генерировала произвольные символы.

In [ ]:
def sample(net, size, prime="The", top_k=None):

    if train_on_gpu:
        net.cuda()
    else:
        net.cpu()

    net.eval()  # eval mode

    # First off, run through the prime characters
    chars = [ch for ch in prime]
    h = net.init_hidden(1)
    for ch in prime:
        char, h = predict(net, ch, h, top_k=top_k)

    chars.append(char)

    # Now pass in the previous character and get a new one
    for ii in range(size):
        char, h = predict(net, chars[-1], h, top_k=top_k)
        chars.append(char)

    return "".join(chars)

In [ ]:
print(sample(net, 1000, prime="Anna", top_k=5))

Anna.

"Why say," thought Stepan Arkadyevitch.

"Why, taken it! Would not let them about her."

"Where is the tenth is awful!" said Vronsky, was still more straight-but her life. "That's it,
and there's no meant of te part," said the legity, stream of
singer. "To take the sound of more sense when the steps, and I hear you with a mere all this
treat of
corcurory too. That's it to make them
impossible, and what does not change it, I'm so many more to them......."

"Why are you than you. He's not beginning to this stand." He was neven a sight of the district, as
though all a letter would be seen her, she was said, while his window and her former strength
they had a man, said the supper. Shim in the supper aroused, were, the preterr of the conversation of this man who was to take
a member, whom she saw that there had been time to bad anyone they would be talking in answer. Anna still there her son, of his wife, sand in the rubble operation to spend he would have taken him,
and that the sen

## Loading a checkpoint

In [ ]:
# Here we have loaded in a model that trained over 20 epochs `rnn_20_epoch.net`
with open("rnn_x_epoch.net", "rb") as f:
    checkpoint = torch.load(f)

loaded = CharRNN(
    checkpoint["tokens"],
    n_hidden=checkpoint["n_hidden"],
    n_layers=checkpoint["n_layers"],
)
loaded.load_state_dict(checkpoint["state_dict"])

<All keys matched successfully>

In [ ]:
# Sample using a loaded model
print(sample(loaded, 2000, top_k=5, prime="And Levin said"))

And Levin said to Vronsky in his heart.

"Yes, I want to see him. Have you, there will be something of the
position," he thought, began to talk to him with the
sungring and an insidely official chores, which he was dead, as he strange
her eyes.

There was a proprising at
all timidle. And
at his horses she felt something but he
could not take them. At a learned, were taking a conversation with his winder, and was a little. The mother were contemptuously as the dream in his shoulders, he wanted.

Stepan Arkadyevitch talked about that the wintow and saw the
staid, and that she was delightful to see her,
which she could
not
see him and so as to say to said. The porter, this stream of the sungen them over with hin sposts and he had no compresent, they was the secrets of
passing and decided by talk of the same,
and so as to do it was not been from the possession as to be thinking of the carriage.

"And what am I show," said Levin to himself. The convistion of the same wife and asked his brot

### Полезные ссылки:


*   [Блог-пост Christopher'а Olah'а по LSTM](https://colah.github.io/posts/2015-08-Understanding-LSTMs/)